# Quantum Phase Estimation

This notebook demonstrates the Qiskit `PhaseEstimation` component.

* [Qiskit HamiltonianGate documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.HamiltonianGate)
* [Qiskit PhaseEstimation documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.PhaseEstimation)

From the documentation:

In the Quantum Phase Estimation (QPE) algorithm, the Phase Estimation circuit is used to estimazte the phase $\phi$ of an eigenvalue $e^{2\pi i \phi}$ of a unitary operator $U$, provided with the corresponding eigenstate $|\psi\rangle$. That is
$$
U|\psi\rangle = e^{2\pi i \phi}|\psi\rangle.
$$
This estimation (eand thereby this circuit) is a central routine to several well-known algorithms, such as Shor's algorithm or Quantum Amplitude Estimation.

In [ ]:
import math
import numpy as np
from IPython.display import display, Markdown
from math import log2
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import HamiltonianGate, PhaseEstimation, QFT, StatePreparation
from qiskit.quantum_info import Statevector
from qiskit.visualization.library import _generate_circuit_library_visualization

π = math.pi
# Pauli matrices: the operators
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Eigenvectors of Z
z0 = np.array([0, 1])
z1 = np.array([1, 0])
# Eigenvectors of Y
y0 = np.array([1, -1j]) / np.sqrt(2)
y1 = np.array([1, 1j]) / np.sqrt(2)

def make_circuit(op, e, apply_qpe=True):
    U = HamiltonianGate(op, -π/2, label='U')
    circuit = QuantumCircuit(3)
    circuit.append(StatePreparation(Statevector(e)), [2])
    if apply_qpe:
        qpe = PhaseEstimation(2, U)
        # _generate_circuit_library_visualization(qpe)
        circuit.append(qpe, [0, 1, 2])
    return circuit

def run_statevector(circuit):
    simulator = Aer.get_backend('statevector_simulator')
    aer_circuit = transpile(circuit, simulator)
    result_statevector = simulator.run(aer_circuit).result().get_statevector()
    return result_statevector

def markdown_statevector(sv):
    return sv.draw('latex').data

def escape_statevector(sv):
    return markdown_statevector(sv).replace('|', r'\|')

data = [
    {'opname': 'Z', 'ename': 'z_0', 'op': Z, 'e': z0},
    {'opname': 'Z', 'ename': 'z_1', 'op': Z, 'e': z1},
    {'opname': 'Y', 'ename': 'y_0', 'op': Y, 'e': y0},
    {'opname': 'Y', 'ename': 'y_1', 'op': Y, 'e': y1},
]

table = "| Operator | Eigenvector | Input statevector | Output statevector |\n"
table += "| --- | --- | --- | --- |\n"
for d in data:
    opname = d['opname']
    ename = d['ename']
    input_statevector = escape_statevector(run_statevector(make_circuit(d['op'], d['e'], apply_qpe=False)))
    output_statevector = escape_statevector(run_statevector(make_circuit(d['op'], d['e'])))
    table += f"| ${opname}$ | ${ename}$ | {input_statevector} | {output_statevector} |\n"
display(Markdown(table))


| Operator | Eigenvector | Input statevector | Output statevector |
| --- | --- | --- | --- |
| $Z$ | $z_0$ | $$ \|100\rangle$$ | $$ \|111\rangle$$ |
| $Z$ | $z_1$ | $$ \|000\rangle$$ | $$ \|010\rangle$$ |
| $Y$ | $y_0$ | $$\frac{\sqrt{2}}{2} \|000\rangle- \frac{\sqrt{2} i}{2} \|100\rangle$$ | $$\frac{\sqrt{2}}{2} \|011\rangle- \frac{\sqrt{2} i}{2} \|111\rangle$$ |
| $Y$ | $y_1$ | $$\frac{\sqrt{2}}{2} \|000\rangle+\frac{\sqrt{2} i}{2} \|100\rangle$$ | $$\frac{\sqrt{2}}{2} \|010\rangle+\frac{\sqrt{2} i}{2} \|110\rangle$$ |
